## Saving ROI data

In [ ]:
VISUALISE = False

In [ ]:
root = '../data/JIGSAWS/'

skills = ['Knot_Tying', 'Needle_Passing', 'Suturing']

cornerNum = 30 #used not just for determining the number of points to select for each feature (each tool)
               #but also for making sure every ROI has all the possible feature points

feature_params = dict( maxCorners = cornerNum,
                       qualityLevel = 0.6,
                       minDistance = 3,
                       blockSize = 3,
                       gradientSize = 5)

import os
import glob
import numpy as np
import cv2 as cv
import pyautogui

def getJIGSAWvideoTitles():
    dir_name = root + skills[0] + '/video/'
    knot_list = [os.path.basename(x) for x in glob.glob(dir_name+'*.avi')]
    dir_name = root + skills[1] + '/video/'
    needle_list = [os.path.basename(x) for x in glob.glob(dir_name+'*.avi')]
    dir_name = root + skills[2] + '/video/'
    suture_list = [os.path.basename(x) for x in glob.glob(dir_name+'*.avi')]
    knot_list.sort()
    needle_list.sort()
    suture_list.sort()
    return knot_list, needle_list, suture_list

knot_list, needle_list, suture_list = getJIGSAWvideoTitles()

def parseFileName(filename):
    var = filename
    index = 0
    skill = var[index] #gathering skill-string
    index += 1
    while(var[index] != '_'):
        skill += var[index]
        index += 1 #finding first underscore
    if filename[0] != 'S': #in case of Suturing there is one less underscore
        skill += var[index]
        index += 1
        while(var[index] != '_'):
            skill += var[index]
            index += 1
    index += 1 #skipping second underscore
    user = var[index] #saving user_id
    index += 1 #skipping userID
    attempt = var[index] #first digit of attempt id
    index += 1
    while(var[index].isdigit()):
        attempt += var[index]
        index += 1
    while(var[index] != '.'):
        index += 1
    capture = var[index-1] #capture_num is right before the dot
    return  user, attempt, capture, skill

def getGestureSeparatedFrameRanges(filename):
    gestures = []
    frame_begin = []
    frame_end = []
    with open(filename) as f:
        lines = f.readlines()
        for line in lines:
            i = 0
            temp = ""
            while line[i] != ' ':
                temp += line[i]
                i += 1
            frame_begin.append(int(temp)) #first characters form the beginning frames
            temp = ""
            while line[i] == ' ':
                i += 1 #skip spaces
            while line[i] != ' ':
                temp += line[i]
                i += 1
            frame_end.append(int(temp)) #second is the end frame
            temp = ""
            while line[i] == ' ':
                i += 1 #skip spaces
            while line[i] != ' ' and line[i] != '\n':
                temp += line[i]
                i += 1
            gestures.append(temp)
    output = np.vstack((np.array(gestures), np.array(frame_begin), np.array(frame_end)))
    f.close()
    return output

def TranscriptionPath(user, attempt, skill):
    path = root + skill + '/transcriptions/'+ skill + '_' + user +  attempt + '.txt'
    return path

def ROIdataPath(user, attempt, skill, capture):
    path = root + skill + '/ROI_data/'+ skill + '_' + user +  attempt +  '_' + capture
    return path

def ChooseROIs(frame, frame_begin, cap):
    num_of_ROIs = 2
    actual_frame_begin = int(frame_begin)
    # Specify a vector of rectangles (ROI) 
    fromCenter = True
    im = cv.medianBlur(frame,5)
    im = cv.adaptiveThreshold(im,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,cv.THRESH_BINARY,11,2)
    # Select multiple rectangles
    i = 0
    while i < num_of_ROIs:
        title = "Image"
        r = cv.selectROI(title, frame, fromCenter)
        if r == (0,0,0,0):
            i = 0
            ret, frame = cap.read()
            frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            actual_frame_begin += 1
        else:
            if VISUALISE:
                cv.imshow('Original',frame)
            im = cv.medianBlur(frame,5)
            if VISUALISE:
                cv.imshow('Blurred', im)
            im = cv.adaptiveThreshold(im,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C,cv.THRESH_BINARY,11,2)
            if VISUALISE:
                cv.imshow('Thresholded', im)
            imCrop = im[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
            img = np.zeros(frame.shape).astype(np.float32)
            img[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])] = imCrop
            if VISUALISE:
                cv.imshow('Cropped area in its original position', img)
                cv.waitKey()
            if i == 0:
                temp1 = []
                features = cv.goodFeaturesToTrack(img, mask = None, **feature_params)
                var = np.count_nonzero(features)
                if  var != 2*cornerNum:
                    i = 0
                    pyautogui.alert("Try again, with a larger area. You had "
                                    + str(var/2) + " out of " + str(cornerNum) + " feature points"
                                    ,title="Not enough corner points within ROI!")
                else:
                    temp1.extend(features)
                    i += 1
            elif i == 1:
                temp2 = []
                features = cv.goodFeaturesToTrack(img, mask = None, **feature_params)
                var = np.count_nonzero(features)
                if var != 2*cornerNum:
                    i = 1
                    pyautogui.alert("Try again, with a larger area. You had "
                                    + str(var/2) + " out of " + str(2*cornerNum) + " feature points"
                                    ,title="Not enough corner points within ROI!")
                else:
                    temp2.extend(features)
                    i += 1
    rects = np.asarray(np.concatenate((temp1, temp2), axis=1)).astype(np.float32)
    cv.destroyWindow(title)
    cap.release()
    return actual_frame_begin, rects


def SaveROIsToDisk(ROI_data, frame_index, skill, user, attempt, capture):
    path = ROIdataPath(user, attempt, skill, capture)
    frame_index_path = path + '_frame_index.txt'
    np.save(path, ROI_data)
    f= open(frame_index_path,"w+")
    f.write(str(frame_index))
    f.close()
    print('Files saved')
    
def getROIDataFromVideoList(vlist, skill):
    path = root + skill + '/video/'
    temp = {}
    i = 0
    v_ind = 0
    for v in vlist:
        v_ind += 1
        user, attempt, capture, skill = parseFileName(v)
        gpath = TranscriptionPath(user, attempt, skill)
        gestureData = getGestureSeparatedFrameRanges(gpath)

        input_path = path + v
        cap = cv.VideoCapture(input_path)
        if not cap.isOpened():
            print('Unable to open: ' + input_path)
            exit(0)

        frame_begin = gestureData[1][0]
        cap.set(cv.CAP_PROP_POS_FRAMES, int(frame_begin)-1)
            
        _, frame1 = cap.read()
        prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
        t = 0
        
        actual_frame_begin, ROIs = ChooseROIs(prvs, frame_begin, cap)
        SaveROIsToDisk(ROIs, actual_frame_begin, skill, user, attempt, capture)
    

def CreateAndSaveROIdata(knots, needles, sutures):
    getROIDataFromVideoList(knots, 'Knot_Tying')
    getROIDataFromVideoList(needles, 'Needle_Passing')
    getROIDataFromVideoList(sutures, 'Suturing')

In [ ]:
CreateAndSaveROIdata(knot_list, needle_list, suture_list)